In [0]:
!pip install pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

import datetime

import numpy as np
import pandas as pd
%matplotlib
from matplotlib import pyplot as plt

In [0]:
path="data/COVID19_ZM_transformedTweets.csv"

spark = SparkSession \
    .builder \
    .appName("COVID-19 Twitter Sentiment Analysis") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [0]:
df = spark.read.csv(path, header=True, sep=",")
print(df.schema)

In [0]:
df = df.withColumn("date", df["created_at"].cast(DateType()))
df = df.withColumn("time", df["created_at"].cast(TimestampType()))
df = df.withColumn("retweet_count", df["retweet_count"].cast(IntegerType()))
df = df.withColumn("user_followers_count", df["user_followers_count"].cast(IntegerType()))
df = df.withColumn("user_followers_count", df["user_followers_count"].cast(IntegerType()))

df.take(5)



In [0]:
df.select("user_screen_name").dropDuplicates().sort("user_screen_name").show()

In [0]:
df=df.withColumn("month", month(df.date))
df.take(5)

In [0]:
df=df.withColumn("day_of_month", dayofmonth(df.date))
df.take(5)

In [0]:
df=df.withColumn("year", year(df.date))
df.take(5)

In [0]:
df=df.withColumn("hour", hour(df.created_at))
df.take(5)

In [0]:
# TODO: get number of tweets per day
num_tweets_per_day=df.filter(df.year==2020).groupBy(df.date).count().orderBy(df.date)
num_tweets_per_day.sort("count",ascending=False).show()

In [0]:
# TODO: who tweets the most about COVID-19
freq_tweeters=df.select(df.user_screen_name).groupBy(df.user_screen_name).count()
freq_tweeters.sort("count",ascending=False).show()

In [0]:
# TODO: Check which hour people tweet the most
hours_with_most_tweets=df.select('*').groupBy(df.hour).count()
hours_with_most_tweets.sort("count",ascending=False).show()

In [0]:
# TODO: Check peoples feeling towards covid
retweets_per_hr=df.select("retweet_count","hour").groupBy("hour").sum("retweet_count")
retweets_per_hr.sort("hour",ascending=False).show()

In [0]:
# TODO: From the people with large following, based on the retwee
df.where(df.hour==23).select("retweet_count","hour").show()

In [0]:
mwebantu_tweets=df.filter(df.user_screen_name=="Mwebantu")
mwebantu_tweets.show()